In [3]:
#pip install pillow selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
import time

In [18]:
# Define the URL of the web page we want to screenshot

url = "https://data.worldbank.org/country/denmark?view=chart"

webdriver_path = 'C:\chromedriver_win32\chromedriver.exe'


# Set up the webdriver

options = webdriver.ChromeOptions()
options.add_argument('--headless=new') 
# Run the browser in headless mode to prevent a window from popping up
driver = webdriver.Chrome(options=options)

In [ ]:
# Load the web page

driver.get(url)
# Take a screenshot of the entire page
try:
    element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    S = lambda X: driver.execute_script("return document.body.parentNode.scroll" + X)
    driver.set_window_size(S('Width'),S('Height'))
    driver.find_element(By.TAG_NAME,"body").screenshot("./saved/denmark.png")
    
finally:
    driver.quit()

In [4]:
img = Image.open("./saved/denmark.png")
img.show()

In [5]:
#######################################
# Step 2: Read text from images
#######################################
# pip install pytesseract

import pytesseract

##############################################################################
# in case if tesseract is not included in PATH
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
##############################################################################

def read_image_text(image_path):
    """
    Reads text from an image file using Tesseract OCR.

    Args:
        image_path (str): The file path to the input image.

    Returns:
        str: The extracted text from the image.
    """
    # Load the image file
    image = Image.open(image_path)

    # Use Tesseract OCR to extract the text from the image
    text = pytesseract.image_to_string(image)

    return text

image_path = "./saved/denmark.png"
text = read_image_text(image_path)
print(text)

© pewonvewm | dace

f¥ DataBank Microdata Data Catalog

Denmark © Jump to ~
Overview By Theme By SDG Goal Graph, map and compare more
— than 1,000 time series indicators

from the World Development
Indicators.
Topic ~ + Income level: High Income
+ Region: Europe & Central Asia
Social
4, Download
Cl csv XML
Indicator Most recent value Trend EXCEL

®@ DataBank

=
Poverty headcount ratio at = Explore Our
$2.15 a day (2017 PPP) (% of. 0.2 Databank
sopulation) © cay)
Ga @ Country Profile
Life expectancy at birth, total 81
(years) O (2021)
Population, total © 5,903,037 =
(2022)
Population growth (annual 0.8
%) © (2022)
Net migration © 22,746 |
(2021) =
Human Capital Index (HCI) 0.8 es
(scale 0-1) © (2020)
Economic
Indicator Most recent value Trend
GDP (current US$) © 395.4
current US$ (2022 billion) ra
GDP per capita (current US$) © 66,983.1 :
[current USS (2022) = |
GDP growth (annual %) © 3.8
(2022)
Unemployment, total (% of total
labor force) (modeled ILO 4.2
estimate) © CEP)
Inflation, c

In [6]:
from decouple import config
import together
from langchain import PromptTemplate, LLMChain
from typing import Any, Dict, List, Mapping, Optional
#pydantic model version>2 will not work with langchain. you should install a model version less than 2
from pydantic import Extra, Field, root_validator

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env
import re

together.api_key = config("TOGETHER_API_KEY")

together.Models.start("togethercomputer/llama-2-70b-chat")

class TogetherLLM(LLM):
    """Together large language models."""

    model = "togethercomputer/llama-2-70b-chat"
    """model endpoint to use"""

    together_api_key = config("TOGETHER_API_KEY")
    """Together API key"""

    temperature = 0.7
    """What sampling temperature to use."""

    max_tokens = 512
    """The maximum number of tokens to generate in the completion."""

    class Config:
        extra = Extra.forbid

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the API key is set."""
        api_key = get_from_dict_or_env(
            values, "together_api_key", "TOGETHER_API_KEY"
        )
        values["together_api_key"] = api_key
        return values

    @property
    def _llm_type(self):
        """Return type of LLM."""
        return "together"

    def _call(
        self,
        prompt,
        **kwargs: Any,
    ):
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        # Use regex substitution to remove newlines
        #cleaned_text = re.sub(r"\n", "", text)
        return text

tog_llm = TogetherLLM(
    model= "togethercomputer/llama-2-70b-chat",
    temperature=0.1,
    max_tokens=1024)

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""


def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template


In [15]:
system_prompt = "You are an advanced assistant. You are always helpful in times of research "
instruction = "how many Individuals are using the Internet?find out from the text : {text}"
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt= prompt, llm = tog_llm )

[INST]<<SYS>>
You are an advanced assistant. You are always helpful in times of research 
<</SYS>>

how many Individuals are using the Internet?find out from the text : {text}[/INST]


In [16]:
output = llm_chain.run(text)

print(output)

 According to the text, the percentage of individuals using the internet in Denmark is 99% (scale 0-100) © (2021).


In [17]:
system_prompt = "You are an advanced assistant. You are always helpful in times of research "
instruction = "Make a table with 3 columns like an excel sheet with border. The columns will represent the indicator, most recent value and trends from the text. you will write the trends are ascending, descending or stable only. try to be accurate. here is the text : {text}"
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt= prompt, llm = tog_llm )

[INST]<<SYS>>
You are an advanced assistant. You are always helpful in times of research 
<</SYS>>

Make a table with 3 columns like an excel sheet with border. The columns will represent the indicator, most recent value and trends from the text. you will write the trends are ascending, descending or stable only. try to be accurate. here is the text : {text}[/INST]


In [19]:
raw_data = llm_chain.run(text)
f = open("raw_data.txt","w")
print(raw_data, file= f) 
f.close()

In [26]:
import pandas as pd
df = pd.read_table("./raw_data.txt",sep="|")
df_denmark = df.drop(["Unnamed: 0","Unnamed: 4"], axis=1)
df_denmark
df_denmark.to_csv("denmark.csv",index= True)
df_denmark.to_excel("denmark.xlsx", index= True)